In [1]:
hf_token = "huggingface-token-here"

In [2]:
from huggingface_hub import notebook_login
notebook_login()

## Mistral 7B v0.3

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm import tqdm
import os
cache_dir = "../cache"
os.environ['TRANSFORMERS_CACHE'] = cache_dir

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda:0", 
    torch_dtype="auto",
    trust_remote_code=True,
    cache_dir=cache_dir, 
    token=hf_token
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
chatbot(messages)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': [{'role': 'system',
    'content': 'You are a pirate chatbot who always responds in pirate speak!'},
   {'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': " Arr matey! I be the digital parrot, ye scurvy dog! I'll be spinnin' ye yarns and crackin' ye jokes in me hearty pirate tongue. Now, what be ye seekin', landlubber?\n\nYe can call me Cap'n Parrot, or just Parrot if ye like. I'll be here to help ye navigate the digital seas and find yer treasure! So, what's"}]}]

In [4]:
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=4, do_sample=False, num_return_sequences=1, return_full_text=False, pad_token_id=2)
chatbot(messages)

[{'generated_text': ' Arr matey!'}]

In [7]:
print(chatbot(messages)[0]['generated_text'])

 Arr matey!


In [17]:
# Function to extract sentiment from model output
def extract_sentiment(output):
    response = output.split(" ")
    if "Positive" or "positive" in response:
        return "Positive"
    elif "Negative" or "negative" in response:
        return "Negative"
    elif "Neutral" or "neutral" in response:
        return "Neutral"
    else:
        return "Unknown"


def generate_sentiment(examples):
    outputs = chatbot(
       [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in African American English and identify the sentiment expressed by the author. The sentiment must be classified as Positive, Negative, or Neutral. Reply with just the sentiment.\n"},
    {"role": "user", "content": f"{examples['text']}"}
]
    )
    return extract_sentiment(outputs[0]['generated_text'])

In [20]:
import datasets
from datasets import Dataset



datasets.logging.set_verbosity_info()
# Load the dataset
df = pd.read_csv('./Complete Dataset/GPT-4o-mini-AAE-sentiment.csv')[["text"]]
# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
# Add prompts to the dataset
dataset = dataset.map(lambda example: {"sentiment": generate_sentiment(example)})

Parameter 'function'=<function <lambda> at 0x7f769c7bc160> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [21]:
dataset[1]

{'text': "@islandboi_B yes that's what's up. Nothin like getting dressed up and getting some culture man.",
 'sentiment': 'Positive'}

In [22]:
# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

# Save the labeled dataset to a CSV file
output_path = './seperate models/Mistral-AAE-Labels.csv'
labeled_df.to_csv(output_path, index=False)

In [23]:
labeled_df['sentiment'].unique()

array(['Negative', 'Positive', 'Neutral', 'Unknown'], dtype=object)

In [28]:
mask = ~labeled_df["sentiment"].isin(["Positive", "Negative", "Neutral"])
labeled_df.loc[mask, "sentiment"] = labeled_df[mask].apply(generate_sentiment, axis=1)

In [30]:
labeled_df[labeled_df['sentiment'] == 'Unknown'].count()

text         8
sentiment    8
dtype: int64

_______________

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("./seperate models/Mistral-AAE-Labels.csv")

In [3]:
dataset['sentiment'].unique()

array(['Negative', 'Positive', 'Neutral', 'Unknown'], dtype=object)

In [13]:
def extract_sentiment(output):
    response = output.split(" ")
    if "Positive" or "positive" in response:
        return "Positive"
    elif "Negative" or "negative" in response:
        return "Negative"
    elif "Neutral" or "neutral" in response:
        return "Neutral"
    else:
        return "Unknown"
    
    
def generate_sentiment(sentence):
    outputs = chatbot(
       [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in African American English and identify the sentiment expressed by the author. The sentiment must be classified as Positive, Negative, or Neutral. Reply with just the sentiment.\n"},
    {"role": "user", "content": f"{sentence}"}
]
    )
    return extract_sentiment(outputs[0]["generated_text"])

# Function to process a single row
def process_row_sentiment(row):
    return generate_sentiment(row["text"])

In [14]:
# Process rows with non-standard sentiments
mask = ~dataset["sentiment"].isin(["Positive", "Negative", "Neutral"])
dataset.loc[mask, "sentiment"] = dataset[mask].apply(process_row_sentiment, axis=1)

In [15]:
dataset['sentiment'].unique()

array(['Negative', 'Positive', 'Neutral'], dtype=object)

In [16]:
dataset.to_csv("./seperate models/Mistral-AAE-Labels.csv", index=False)

## Translation to SAE

In [18]:
dataset["text"].to_csv("./seperate models/5000_AAE_sentence.csv", index=False)

In [22]:
df = dataset.copy()

In [29]:
import datasets
from datasets import Dataset

datasets.logging.set_verbosity_info()

# Function to create prompts for sentiment analysis
def create_prompt(example):
    return  [
    {"role": "system", "content": "Following is a tweet extracted from a African American twitter individual's account. Your task is to convert the tweet to Standard American English. Reply with just the sentence."},
    {"role": "user", "content": f"{example['text']}"}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset
dataset = dataset.map(lambda example: {"prompt": create_prompt(example)})

# Function to generate sentiments
def generate_sae(examples):
    outputs = chatbot(
      examples["prompt"]
    )
    return {"generated_text": [output[0]["generated_text"] for output in outputs]}

# Generate sentiments
dataset = dataset.map(
    generate_sae,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()



Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Parameter 'function'=<function generate_sae at 0x7ff6800f6310> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

successfully generated models answers


KeyError: "['standard_american_english'] not in index"

In [32]:
# Save the labeled dataset to a CSV file
output_path = './seperate models/Mistral-SAE.csv'
labeled_df[["text", "generated_text"]].rename(columns={"text": "african_american_english", "generated_text" : "standard_american_english"}).to_csv(output_path, index=False)

print(f"SAE dataset saved to {output_path}")
print(f"Number of processed sentences: {len(labeled_df)}")

SAE dataset saved to ./seperate models/Mistral-SAE.csv
Number of processed sentences: 5000


# Getting labels for SAE

In [8]:
df = pd.read_csv('./seperate models/Mistral-SAE.csv')

In [10]:
import datasets
from datasets import Dataset

datasets.logging.set_verbosity_info()


# Function to create prompts for sentiment analysis
def create_prompt(example):
    return  [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in Standard American English and identify the sentiment expressed by the author. Reply back in one word with just the sentiment, Positive, Negative, or Neutral only\n"},
    {"role": "user", "content": f"{example['standard_american_english']}"}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset
dataset = dataset.map(lambda example: {"prompt": create_prompt(example)})

def extract_sentiment(output):
    response = output.split(" ")
    if "Positive" in response:
        return "Positive"
    elif "Negative"  in response:
        return "Negative"
    elif "Neutral" in response:
        return "Neutral"
    else:
        return "Unknown"

# Function to generate sentiments
def generate_sae_labels(examples):
    outputs = chatbot(
      examples["prompt"]
    )
    return {"sae_labels": [extract_sentiment(output[0]["generated_text"]) for output in outputs],
           "model_generation" : [output[0]["generated_text"] for output in outputs]}

# Generate sentiments
dataset = dataset.map(
    generate_sae_labels,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

output_path = './seperate models/Mistral-SAE-sentiment.csv'
labeled_df[["standard_american_english", "sae_labels", "model_generation"]].to_csv(output_path, index=False)

print(f"SAE dataset saved to {output_path}")
print(f"Number of processed sentences: {len(labeled_df)}")

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Parameter 'function'=<function generate_sae_labels at 0x7f9e1ec00280> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


successfully generated models answers
SAE dataset saved to ./seperate models/Mistral-SAE-sentiment.csv
Number of processed sentences: 5000


In [35]:
def generate_sentiment(sentence):
    outputs = chatbot(
      text_inputs=  [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in Standard American English and identify the sentiment expressed by the author. The sentiment must be classified as Positive, Negative, or Neutral. Reply with just the sentiment.\n\n"},
    {"role": "user", "content": f"{sentence}"}
],
        max_new_tokens=15,
        do_sample=False,
        num_return_sequences=1
    )
    return extract_sentiment(outputs[0]["generated_text"])

In [36]:
# Function to process a single row
def process_row_sentiment(row):
    return generate_sentiment(row['standard_american_english'])

In [37]:
test = labeled_df.copy()

In [38]:
# Process rows with non-standard sentiments
mask = ~test["sae_labels"].isin(["Positive", "Negative", "Neutral"])
test.loc[mask, "sae_labels"] = test[mask].apply(process_row_sentiment, axis=1)

In [43]:
test['sae_labels'].value_counts()

sae_labels
Negative    2218
Positive    1881
Neutral      901
Name: count, dtype: int64

In [40]:
test[test['sae_labels'] == 'Unknown']

,african_american_english,standard_american_english,prompt,sae_labels,model_generation
1905,Ima stop listening to music and go on nexflix....,I will stop listening to music and switch to ...,[{'content': 'Your task is to analyze the prov...,Unknown,"Neutral,"


In [41]:
test.loc[test['sae_labels'] == 'Unknown', 'sae_labels'] = 'Neutral'

In [42]:
test[test['sae_labels'] == 'Unknown']

,african_american_english,standard_american_english,prompt,sae_labels,model_generation


In [44]:
output_path = './seperate models/Mistral-SAE-sentiment.csv'
test[["standard_american_english", "sae_labels"]].to_csv(output_path, index=False)

## Converting back to AAE

In [5]:
df = pd.read_csv('./seperate models/Mistral-SAE.csv')

In [6]:
import datasets
from datasets import Dataset

datasets.logging.set_verbosity_info()

# Function to create prompts for sentiment analysis
def create_aae_from_sae_prompt(example):
    return  [
    {"role": "system", "content": "You will be given a tweet in Standard American English. Your task is to convert the given tweet to African American English. Reply with just the translated sentence."},
    {"role": "user", "content": f"{example['standard_american_english']}"}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset
dataset = dataset.map(lambda example: {"prompt": create_aae_from_sae_prompt(example)})

# Function to generate sentiments
def generate_aae_from_sae(examples):
    outputs = chatbot(
      examples["prompt"],
    max_new_tokens=150,
        do_sample=False,
        num_return_sequences=1
    )
    return {"AAE_from_SAE": [output[0]["generated_text"] for output in outputs]}

# Generate sentiments
dataset = dataset.map(
    generate_aae_from_sae,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

output_path = './seperate models/Mistral-AAE_from_SAE.csv'
labeled_df[["standard_american_english", "AAE_from_SAE"]].to_csv(output_path, index=False)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Parameter 'function'=<function generate_aae_from_sae at 0x7f8f212e3ee0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


successfully generated models answers


## AAE_from_SAE sentiment

In [9]:
# Function to create prompts for sentiment analysis
def create_prompt_aae_from_sae(example):
    return  [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in African American English and identify the sentiment expressed by the author. The sentiment must be classified as Positive, Negative, or Neutral. Reply with just the sentiment.\n"},
    {"role": "user", "content": f"{example['AAE_from_SAE']}"}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(labeled_df[["AAE_from_SAE"]])

# Add prompts to the dataset
dataset = dataset.map(lambda example: {"prompt": create_prompt_aae_from_sae(example)})

# Function to extract sentiment from model output
def extract_sentiment(response):
    if "Positive"  in response:
        return "Positive"
    elif "Negative"  in response:
        return "Negative"
    elif "Neutral" in response:
        return "Neutral"
    else:
        return "Unknown"


def generate_sentiment_aae_from_sae(examples):
    outputs = chatbot(
       examples['prompt']
    )
    return {"AAE_from_SAE-sentiment" : [extract_sentiment(output[0]["generated_text"]) for output in outputs]}

# Generate sentiments
dataset = dataset.map(
    generate_sentiment_aae_from_sae,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

output_path = './seperate models/Mistral-AAE_from_SAE-sentiment.csv'
labeled_df.to_csv(output_path, index=False)

print(f"AAE_from_SAE-sentiment dataset saved to {output_path}")
print(f"Number of processed sentences: {len(labeled_df)}")

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Parameter 'function'=<function generate_sentiment_aae_from_sae at 0x7f8ae16309d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

successfully generated models answers
AAE_from_SAE-sentiment dataset saved to ./seperate models/Mistral-AAE_from_SAE-sentiment.csv
Number of processed sentences: 5000


In [10]:
labeled_df[["AAE_from_SAE", "AAE_from_SAE-sentiment"]].to_csv(output_path, index=False)

In [2]:
import pandas as pd
already_labeled = pd.read_csv("./seperate models/Mistral-AAE_from_SAE-sentiment.csv") 

In [3]:
already_labeled["AAE_from_SAE-sentiment"].unique()

array(['Negative', 'Positive', 'Neutral'], dtype=object)